In [ ]:
import lief
import pefile
import shutil
import polars as pl

In [ ]:
path = r"G:\Games\FA\FA-EMU\Shipping\GAME_dump.exe"
original_path = r"G:\Games\FA\FA-EMU\Shipping\GAME.exe"
out_path = r"G:\Games\FA\FA-EMU\Shipping\GAME_dump_mod.exe"

shutil.copy(path, out_path)

In [ ]:
pe = lief.PE.parse(out_path)
assert pe is not None
pe.remove_all_imports()

In [ ]:
base = "../game-dump/"
fn_new_idt = base + "new_idt.csv"
idt = pl.read_csv(fn_new_idt)

In [ ]:
def create_32bit_ordinal_import(ordinal_number: int) -> lief.PE.ImportEntry:
    """
    Create a 32-bit import by ordinal

    Args:
        ordinal_number: The ordinal number (0-65535)
    """
    # Validate ordinal range
    if ordinal_number < 0 or ordinal_number > 0xFFFF:
        raise ValueError("Ordinal number must be between 0 and 65535")

    # For 32-bit PE:
    # - Set bit 31 to 1 (0x80000000)
    # - Bits 30-16 must be 0
    # - Bits 15-0 contain the ordinal
    ORDINAL_MASK_32 = 0x80000000
    data_value = ORDINAL_MASK_32 | ordinal_number

    # Create the import entry
    entry = lief.PE.ImportEntry(data_value, lief.PE.PE_TYPE.PE32)

    return entry

In [ ]:
last_mod_name = None
last_mod = None

for addr, dll, func in idt.rows():
    if dll == last_mod_name:
        mod = last_mod
    else:
        mod = pe.add_import(dll)
        if dll == "msvcp90.dll":
            print(mod)

    if func.startswith("Ordinal#"):
        ordinal = int(func.removeprefix("Ordinal#"))
        print(
            "made by ordinal",
            ordinal,
            mod.add_entry(create_32bit_ordinal_import(ordinal)),
        )
    else:
        mod.add_entry(func)

    last_mod = mod
    last_mod_name = mod.name

In [ ]:
config = lief.PE.Builder.config_t()
config.imports = True

bb = lief.PE.Builder(pe, config)
bb.build()
bb.write(out_path)

## Troubleshooting

In [ ]:
pef = pefile.PE(out_path)
pef.parse_data_directories()

In [ ]:
i = 0
imports: dict[str, list[str]] = {}

for entry in pef.DIRECTORY_ENTRY_IMPORT:
    mod = entry.dll.decode()
    for impo in entry.imports:
        if impo.name:
            name = impo.name.decode()
        else:
            name = f"Ordinal#{impo.ordinal}"

        if mod not in imports:
            imports[mod] = []
        imports[mod].append(name)
        i += 1
print(i, idt.shape[0])

In [ ]:
not_found = 0
for addr, dll, func in idt.rows():
    if dll not in imports:
        print(f"[!] Module {dll} not found in PE!")
        not_found += 1
        continue

    if func not in imports[dll]:
        print(f"[!] Func {func} from {dll} not found in PE!")
        not_found += 1
        continue

print("Not found", not_found)

In [ ]:
idt_unique = idt.with_columns(
    (pl.col("Module").shift(1) != pl.col("Module")).alias("next")
).fill_null(True)
idt_unique = idt_unique.filter(pl.col("next")).drop("next")

i = 0
for addr, dll, func in idt_unique.rows():
    while pef.DIRECTORY_ENTRY_IMPORT[i].dll.decode() != dll:
        i += 1
        if i >= len(pef.DIRECTORY_ENTRY_IMPORT):
            break
    assert i < len(pef.DIRECTORY_ENTRY_IMPORT)

    pef.DIRECTORY_ENTRY_IMPORT[i].struct.FirstThunk = (
        int(addr, 16) - pef.OPTIONAL_HEADER.ImageBase
    )


In [ ]:
temp = "exe.exe"
pef.write(filename=temp)

In [ ]:
pef.close()
shutil.move(temp, out_path)

In [ ]:
pl.Config(tbl_rows=40)

In [ ]:
idt_unique.with_columns(
    (
        pl.col("Address").str.slice(2).str.to_integer(base=16)
        - pef.OPTIONAL_HEADER.ImageBase
    ).map_elements(lambda x: hex(x)[2:], return_dtype=pl.Utf8)
)